In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()



#groq_api_key=os.getenv("GROQ_API_KEY")
#groq_api_key

In [ ]:
from langchain_groq import ChatGroq
#llm=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
llm

In [ ]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader=WebBaseLoader("https://www.technia.com/en/user-experience/software/value-components/")
loader

In [ ]:
docs=loader.load()
docs

In [ ]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [ ]:
documents

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [ ]:
from langchain_chroma import Chroma
vectordb=Chroma.from_documents(documents=documents,embedding=embeddings,persist_directory="./chroma_db")

In [ ]:
result = vectordb.similarity_search("What are value components?")
result[0].page_content

In [ ]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
{input}
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

In [ ]:
retriever=vectordb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
config={"run_name": "personal-webbased-rag", "tags": ["rag"], "metadata": {"model": "groq-gemma"}}

In [ ]:
response=retrieval_chain.invoke({"input":"What are features of value components?"},config=config)
response['answer']